# chapter 6 Pandasを使ったデータ加工処理
# 6-1 概要と事前準備
## 6-1-1 この章で使用するライブラリのインポート

In [500]:
import numpy as np
import numpy.random as random
import scipy as sp
import pandas as pd
from pandas import Series, DataFrame

#　可視化ライブラリ
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

%precision 3

'%.3f'

In [501]:
# 6-2 Pandasの基本的なデータ操作
## 6-2-1 階層型インデックス

In [502]:
hier_df = DataFrame(
	np.arange(9).reshape((3,3)),
	index = [
		['a', 'a', 'b'],
		[1,2,2]
	],
	columns=[
		['Osaka', 'Tokyo', 'Osaka'],
		['Blue', 'Red', 'Red']
	]
)
hier_df

Osaka Tokyo Osaka
     Blue   Red   Red
a 1     0     1     2
  2     3     4     5
b 2     6     7     8

In [503]:
# indexに名前をつける
hier_df.index.names = ['key1', 'key2']

# カラムに名前をつける
hier_df.columns.names = ['city', 'color']
hier_df

city      Osaka Tokyo Osaka
color      Blue   Red   Red
key1 key2                  
a    1        0     1     2
     2        3     4     5
b    2        6     7     8

### 1-1 カラムの絞り込み

In [504]:
hier_df['Osaka']

color      Blue  Red
key1 key2           
a    1        0    2
     2        3    5
b    2        6    8

### 1-2 インデックスを軸にした集計

In [505]:
# 階層ごとの要約統計量：行合計（axis=0は省略可能）
hier_df.groupby(level='key2').sum()

city,Osaka,Tokyo,Osaka
color,Blue,Red,Red
key2,,,
1,0,1,2
2,9,11,13


In [506]:
# 階層ごとの要約統計量：列合計
hier_df.groupby(level='color', axis=1).sum()

/var/folders/5k/0bfc6pzn3z9_z6xn1rvv40sr0000gn/T/ipykernel_74642/2918113267.py:2: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  hier_df.groupby(level='color', axis=1).sum()


color      Blue  Red
key1 key2           
a    1        0    3
     2        3    9
b    2        6   15

### 1-3 インデックス要素の削除

In [507]:
hier_df.drop(['b'])

city      Osaka Tokyo Osaka
color      Blue   Red   Red
key1 key2                  
a    1        0     1     2
     2        3     4     5

### 練習問題 6-1 

In [508]:
hier_df1 = DataFrame(
	np.arange(12).reshape(3,4),
	index = [['c','d','d'], [1,2,1]],
	columns=[
		['Kyoto', 'Nagoya', 'Hokkaido', 'Kyoto'],
		['Yellow', ' Yellow', 'Red', 'Blue']
	]
)

hier_df1.index.names = ['key1', 'key2']
hier_df1.columns.names = ['city', 'color']
hier_df1

city       Kyoto  Nagoya Hokkaido Kyoto
color     Yellow  Yellow      Red  Blue
key1 key2                              
c    1         0       1        2     3
d    2         4       5        6     7
     1         8       9       10    11

### 練習問題 6-2

In [509]:
hier_df1.groupby(level='city', axis=1).mean()

/var/folders/5k/0bfc6pzn3z9_z6xn1rvv40sr0000gn/T/ipykernel_74642/1192816388.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  hier_df1.groupby(level='city', axis=1).mean()


city       Hokkaido  Kyoto  Nagoya
key1 key2                         
c    1          2.0    1.5     1.0
d    2          6.0    5.5     5.0
     1         10.0    9.5     9.0

### 練習問題 6-3

In [510]:
hier_df1.groupby(level='key2', axis=0).sum()

/var/folders/5k/0bfc6pzn3z9_z6xn1rvv40sr0000gn/T/ipykernel_74642/2007925571.py:1: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  hier_df1.groupby(level='key2', axis=0).sum()


city,Kyoto,Nagoya,Hokkaido,Kyoto
color,Yellow,Yellow,Red,Blue
key2,,,,
1,8,10,12,14
2,4,5,6,7


## 6-2-2 データの結合

In [511]:
# データ1の準備
data1 = {
	'id':['100','101','102','103','104','106','108','110','111','113'],
	'city':['Tokyo', 'Osaka', 'Kyoto', 'Hokkaido', 'Tokyo', 'Tokyo', 'Osaka', 'Kyoto', 'Hokkaido', 'Tokyo'],
	'birth_year':[1990, 1989, 1992, 1997, 1982, 1991, 1988, 1990, 1995, 1981],
	'name':['Hiroshi', 'Akiko', 'Yuki', 'Satoru', 'Steeve', 'Mituru', 'Aoi', 'Tarou', 'Suguru', 'Mitsuo']
}
df1 = DataFrame(data1)
df1

,id,city,birth_year,name
0,100,Tokyo,1990,Hiroshi
1,101,Osaka,1989,Akiko
2,102,Kyoto,1992,Yuki
3,103,Hokkaido,1997,Satoru
4,104,Tokyo,1982,Steeve
5,106,Tokyo,1991,Mituru
6,108,Osaka,1988,Aoi
7,110,Kyoto,1990,Tarou
8,111,Hokkaido,1995,Suguru
9,113,Tokyo,1981,Mitsuo


In [512]:
# データ2の準備
data2 = {
	'id':['100', '101', '102', '105', '107'],
	'math':[50, 43, 33, 76, 98],
	'english':[90, 30, 20, 50, 30],
	'sex':['M', 'F', 'F', 'M', 'M'],
	'index_num':[0,1,2,3,4]
}

df2 = DataFrame(data2)
df2

,id,math,english,sex,index_num
0,100,50,90,M,0
1,101,43,30,F,1
2,102,33,20,F,2
3,105,76,50,M,3
4,107,98,30,M,4


### 2-1 結合
### 2-2 内部結合

In [513]:
# mergeは自動的に内部結合になる（how='inner'）
# キーは自動で認識されるが、onで明示的に指定可能
pd.merge(df1, df2, on='id')

,id,city,birth_year,name,math,english,sex,index_num
0,100,Tokyo,1990,Hiroshi,50,90,M,0
1,101,Osaka,1989,Akiko,43,30,F,1
2,102,Kyoto,1992,Yuki,33,20,F,2


### 2-3 全結合

In [514]:
pd.merge(df1,df2,how='outer')

,id,city,birth_year,name,math,english,sex,index_num
0,100,Tokyo,1990.0,Hiroshi,50.0,90.0,M,0.0
1,101,Osaka,1989.0,Akiko,43.0,30.0,F,1.0
2,102,Kyoto,1992.0,Yuki,33.0,20.0,F,2.0
3,103,Hokkaido,1997.0,Satoru,NaN,NaN,NaN,NaN
4,104,Tokyo,1982.0,Steeve,NaN,NaN,NaN,NaN
5,105,NaN,NaN,NaN,76.0,50.0,M,3.0
6,106,Tokyo,1991.0,Mituru,NaN,NaN,NaN,NaN
7,107,NaN,NaN,NaN,98.0,30.0,M,4.0
8,108,Osaka,1988.0,Aoi,NaN,NaN,NaN,NaN
9,110,Kyoto,1990.0,Tarou,NaN,NaN,NaN,NaN


In [515]:
# indexによるマージ
# left_index =True：左側のデータフレームのindexを元にマージ
# right_on = 'index_num' で右側のデータの'index_num'カラムをキーとして指定
pd.merge(df1,df2, left_index=True, right_on = 'index_num')

,id_x,city,birth_year,name,id_y,math,english,sex,index_num
0,100,Tokyo,1990,Hiroshi,100,50,90,M,0
1,101,Osaka,1989,Akiko,101,43,30,F,1
2,102,Kyoto,1992,Yuki,102,33,20,F,2
3,103,Hokkaido,1997,Satoru,105,76,50,M,3
4,104,Tokyo,1982,Steeve,107,98,30,M,4


### 2-4 左外部結合

In [516]:
pd.merge(df1,df2,how='left')

,id,city,birth_year,name,math,english,sex,index_num
0,100,Tokyo,1990,Hiroshi,50.0,90.0,M,0.0
1,101,Osaka,1989,Akiko,43.0,30.0,F,1.0
2,102,Kyoto,1992,Yuki,33.0,20.0,F,2.0
3,103,Hokkaido,1997,Satoru,NaN,NaN,NaN,NaN
4,104,Tokyo,1982,Steeve,NaN,NaN,NaN,NaN
5,106,Tokyo,1991,Mituru,NaN,NaN,NaN,NaN
6,108,Osaka,1988,Aoi,NaN,NaN,NaN,NaN
7,110,Kyoto,1990,Tarou,NaN,NaN,NaN,NaN
8,111,Hokkaido,1995,Suguru,NaN,NaN,NaN,NaN
9,113,Tokyo,1981,Mitsuo,NaN,NaN,NaN,NaN


### 2-5 縦結合

In [517]:
# データ3の準備
data3 = {
	'id':['117', '118', '119', '120', '125'],
	'city':['Chiba', 'Kanagawa', 'tokyo', 'Fukuoka', 'Okinawa'],
	'birth_year':[1990, 1989, 1992, 1997, 1982],
	'name':['Suguru', 'Kouichi', 'Satochi', 'Yukie', 'Akari']
}
df3 = DataFrame(data3)
df3

,id,city,birth_year,name
0,117,Chiba,1990,Suguru
1,118,Kanagawa,1989,Kouichi
2,119,tokyo,1992,Satochi
3,120,Fukuoka,1997,Yukie
4,125,Okinawa,1982,Akari


In [518]:
# concat 縦結合
concat_data = pd.concat([df1, df3])
concat_data

,id,city,birth_year,name
0,100,Tokyo,1990,Hiroshi
1,101,Osaka,1989,Akiko
2,102,Kyoto,1992,Yuki
3,103,Hokkaido,1997,Satoru
4,104,Tokyo,1982,Steeve
5,106,Tokyo,1991,Mituru
6,108,Osaka,1988,Aoi
7,110,Kyoto,1990,Tarou
8,111,Hokkaido,1995,Suguru
9,113,Tokyo,1981,Mitsuo


### 練習問題 6-4

In [519]:
# データ4の準備
data4 = {
	'id':['0','1','2','3','4','6','8','10','11','13'],
	'city':['Tokyo', 'Osaka', 'Kyoto', 'Hokkaido', 'Tokyo', 'Tokyo', 'Osaka', 'Kyoto', 'Hokkaido', 'Tokyo'],
	'birth_year':[1990, 1989, 1992, 1997, 1982, 1991, 1988, 1990, 1995, 1981],
	'name':['Hiroshi', 'Akiko', 'Yuki', 'Satoru', 'Steeve', 'Mituru', 'Aoi', 'Tarou', 'Suguru', 'Mitsuo']
}
df4 = DataFrame(data4)
df4

,id,city,birth_year,name
0,0,Tokyo,1990,Hiroshi
1,1,Osaka,1989,Akiko
2,2,Kyoto,1992,Yuki
3,3,Hokkaido,1997,Satoru
4,4,Tokyo,1982,Steeve
5,6,Tokyo,1991,Mituru
6,8,Osaka,1988,Aoi
7,10,Kyoto,1990,Tarou
8,11,Hokkaido,1995,Suguru
9,13,Tokyo,1981,Mitsuo


In [520]:
# データ5の準備
data5 = {
	'id':['0', '1', '3', '6', '8'],
	'math':[20, 30, 50,70, 90],
	'english':[20, 30, 50,70, 20],
	'sex':['M', 'F', 'F', 'M', 'M'],
	'index_num':[0,1,2,3,4]
}

df5 = DataFrame(data5)
df5

,id,math,english,sex,index_num
0,0,20,20,M,0
1,1,30,30,F,1
2,3,50,50,F,2
3,6,70,70,M,3
4,8,90,20,M,4


In [521]:
df6 = pd.merge(df4, df5)
df6

,id,city,birth_year,name,math,english,sex,index_num
0,0,Tokyo,1990,Hiroshi,20,20,M,0
1,1,Osaka,1989,Akiko,30,30,F,1
2,3,Hokkaido,1997,Satoru,50,50,F,2
3,6,Tokyo,1991,Mituru,70,70,M,3
4,8,Osaka,1988,Aoi,90,20,M,4


### 練習問題 6-5 

In [522]:
df7 = pd.merge(df4, df5, how='outer')
df7

,id,city,birth_year,name,math,english,sex,index_num
0,0,Tokyo,1990,Hiroshi,20.0,20.0,M,0.0
1,1,Osaka,1989,Akiko,30.0,30.0,F,1.0
2,10,Kyoto,1990,Tarou,NaN,NaN,NaN,NaN
3,11,Hokkaido,1995,Suguru,NaN,NaN,NaN,NaN
4,13,Tokyo,1981,Mitsuo,NaN,NaN,NaN,NaN
5,2,Kyoto,1992,Yuki,NaN,NaN,NaN,NaN
6,3,Hokkaido,1997,Satoru,50.0,50.0,F,2.0
7,4,Tokyo,1982,Steeve,NaN,NaN,NaN,NaN
8,6,Tokyo,1991,Mituru,70.0,70.0,M,3.0
9,8,Osaka,1988,Aoi,90.0,20.0,M,4.0


### 練習問題 6-6

In [523]:
df7 = pd.concat([df5, df6])
df7

,id,math,english,sex,index_num,city,birth_year,name
0,0,20,20,M,0,NaN,NaN,NaN
1,1,30,30,F,1,NaN,NaN,NaN
2,3,50,50,F,2,NaN,NaN,NaN
3,6,70,70,M,3,NaN,NaN,NaN
4,8,90,20,M,4,NaN,NaN,NaN
0,0,20,20,M,0,Tokyo,1990.0,Hiroshi
1,1,30,30,F,1,Osaka,1989.0,Akiko
2,3,50,50,F,2,Hokkaido,1997.0,Satoru
3,6,70,70,M,3,Tokyo,1991.0,Mituru
4,8,90,20,M,4,Osaka,1988.0,Aoi


## 6-2-3 データの操作と変換
### 3-1 ピボット操作

In [524]:
hier_df = DataFrame(
	np.arange(9).reshape((3,3)),
	index =[
		['a', 'a', 'b'],
		[1,2,2]
	],
	columns=[
		['Osaka', 'Tokyo', 'Osaka'],
		['Blue', 'Red', 'Red']
	]
)
hier_df

Osaka Tokyo Osaka
     Blue   Red   Red
a 1     0     1     2
  2     3     4     5
b 2     6     7     8

In [525]:
# ピボット操作で「Blue,Red」の列を行に変更
hier_df.stack()

/var/folders/5k/0bfc6pzn3z9_z6xn1rvv40sr0000gn/T/ipykernel_74642/3354912078.py:2: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  hier_df.stack()


Osaka  Tokyo
a 1 Blue      0    NaN
    Red       2    1.0
  2 Blue      3    NaN
    Red       5    4.0
b 2 Blue      6    NaN
    Red       8    7.0

In [526]:
# unstackメソッドで"Blue, Red"の行を列に変更
hier_df.unstack()
hier_df

Osaka Tokyo Osaka
     Blue   Red   Red
a 1     0     1     2
  2     3     4     5
b 2     6     7     8

### 3-2 重複データの削除

In [527]:
# 重複があるデータを準備
dupli_data = DataFrame({
	'col1':[1,1,2,3,4,4,6,6],
	'col2':['a', 'b', 'b', 'b', 'c', 'c', 'd', 'd']
})
dupli_data

,col1,col2
0,1,a
1,1,b
2,2,b
3,3,b
4,4,c
5,4,c
6,6,d
7,6,d


In [528]:
# 重複判定
dupli_data.duplicated()

0    False
1    False
2    False
3    False
4    False
5     True
6    False
7     True
dtype: bool

In [529]:
#重複削除
dupli_data.drop_duplicates()

,col1,col2
0,1,a
1,1,b
2,2,b
3,3,b
4,4,c
6,6,d


### 3-3 マッピング処理

In [530]:
# 参照データ
city_map = {
	'Tokyo':'Kanto',
	'Hokkaido':'Hokkaido',
	'Osaaka':'Kansai',
	'Kyoto':'Kansai'
}
city_map

{'Tokyo': 'Kanto',
 'Hokkaido': 'Hokkaido',
 'Osaaka': 'Kansai',
 'Kyoto': 'Kansai'}

In [531]:
df1.head()

,id,city,birth_year,name
0,100,Tokyo,1990,Hiroshi
1,101,Osaka,1989,Akiko
2,102,Kyoto,1992,Yuki
3,103,Hokkaido,1997,Satoru
4,104,Tokyo,1982,Steeve


In [532]:
# 参照データを結合
# もし対応するデータがなかったらNaNになる
df1['region'] = df1['city'].map(city_map)
df1

,id,city,birth_year,name,region
0,100,Tokyo,1990,Hiroshi,Kanto
1,101,Osaka,1989,Akiko,NaN
2,102,Kyoto,1992,Yuki,Kansai
3,103,Hokkaido,1997,Satoru,Hokkaido
4,104,Tokyo,1982,Steeve,Kanto
5,106,Tokyo,1991,Mituru,Kanto
6,108,Osaka,1988,Aoi,NaN
7,110,Kyoto,1990,Tarou,Kansai
8,111,Hokkaido,1995,Suguru,Hokkaido
9,113,Tokyo,1981,Mitsuo,Kanto


### 3-4 無名関数とmapを組み合わせる

In [533]:
# birth_yearの上３つの数字・文字を取り出す
# str()で文字列に変換したのちに、[0:3]でスライス
df1['up_two_num'] = df1['birth_year'].map(lambda x: str(x)[0:3])
df1

,id,city,birth_year,name,region,up_two_num
0,100,Tokyo,1990,Hiroshi,Kanto,199
1,101,Osaka,1989,Akiko,NaN,198
2,102,Kyoto,1992,Yuki,Kansai,199
3,103,Hokkaido,1997,Satoru,Hokkaido,199
4,104,Tokyo,1982,Steeve,Kanto,198
5,106,Tokyo,1991,Mituru,Kanto,199
6,108,Osaka,1988,Aoi,NaN,198
7,110,Kyoto,1990,Tarou,Kansai,199
8,111,Hokkaido,1995,Suguru,Hokkaido,199
9,113,Tokyo,1981,Mitsuo,Kanto,198


### 3-5 ビン分割

In [534]:
#分割の粒度
birth_year_bins = [1980, 1985, 1990, 1995, 2000]

# ビン分割の実施
birth_year_cut_data = pd.cut(df1.birth_year, birth_year_bins)
birth_year_cut_data

0    (1985, 1990]
1    (1985, 1990]
2    (1990, 1995]
3    (1995, 2000]
4    (1980, 1985]
5    (1990, 1995]
6    (1985, 1990]
7    (1985, 1990]
8    (1990, 1995]
9    (1980, 1985]
Name: birth_year, dtype: category
Categories (4, interval[int64, right]): [(1980, 1985] < (1985, 1990] < (1990, 1995] < (1995, 2000]]

In [535]:
# 集計結果
pd.value_counts(birth_year_cut_data)

/var/folders/5k/0bfc6pzn3z9_z6xn1rvv40sr0000gn/T/ipykernel_74642/3209956500.py:2: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(birth_year_cut_data)


birth_year
(1985, 1990]    4
(1990, 1995]    3
(1980, 1985]    2
(1995, 2000]    1
Name: count, dtype: int64

In [536]:
# 名前をつける
group_names = ['early1980s', 'late1980s', 'early1990s', 'late1990s']
birth_year_cut_data = pd.cut(df1.birth_year, birth_year_bins, labels=group_names)
pd.value_counts(birth_year_cut_data)

/var/folders/5k/0bfc6pzn3z9_z6xn1rvv40sr0000gn/T/ipykernel_74642/3247626296.py:4: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(birth_year_cut_data)


birth_year
late1980s     4
early1990s    3
early1980s    2
late1990s     1
Name: count, dtype: int64

In [537]:
# 数字で分割可能。ここでは二つに分割。
pd.cut(df1.birth_year, 2)

0      (1989.0, 1997.0]
1    (1980.984, 1989.0]
2      (1989.0, 1997.0]
3      (1989.0, 1997.0]
4    (1980.984, 1989.0]
5      (1989.0, 1997.0]
6    (1980.984, 1989.0]
7      (1989.0, 1997.0]
8      (1989.0, 1997.0]
9    (1980.984, 1989.0]
Name: birth_year, dtype: category
Categories (2, interval[float64, right]): [(1980.984, 1989.0] < (1989.0, 1997.0]]

In [538]:
pd.value_counts(pd.cut(df1.birth_year, 2))

/var/folders/5k/0bfc6pzn3z9_z6xn1rvv40sr0000gn/T/ipykernel_74642/2612910515.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(pd.cut(df1.birth_year, 2))


birth_year
(1989.0, 1997.0]      6
(1980.984, 1989.0]    4
Name: count, dtype: int64

In [539]:
# qcut関数による分位点での分割も可能。
pd.value_counts(pd.qcut(df1.birth_year, 2))

/var/folders/5k/0bfc6pzn3z9_z6xn1rvv40sr0000gn/T/ipykernel_74642/3260044462.py:2: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(pd.qcut(df1.birth_year, 2))


birth_year
(1980.999, 1990.0]    6
(1990.0, 1997.0]      4
Name: count, dtype: int64

### 練習問題 6-7

In [540]:
# 3章でも使用した数学の点数のデータを使用
import requests, zipfile
from io import StringIO
import io

#データがあるurlの指定
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00356/student.zip'

#データをurlから取得する
r = requests.get(url, stream=True)

#zipファイルを読み書きする
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

#データをDataframeとして読み込む
student_data_math = pd.read_csv('student-mat.csv')
#区切りに;がついているので要注意
student_data_math = pd.read_csv('student-mat.csv', sep=';')
student_data_math.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [541]:
# 新たに年齢の2倍の値を入れるカラムを作成
student_data_math['dbl_age'] = student_data_math['age'] * 2
student_data_math.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,dbl_age
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,3,4,1,1,3,6,5,6,6,36
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,3,3,1,1,3,4,5,5,6,34
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,3,2,2,3,3,10,7,8,10,30
3,GP,F,15,U,GT3,T,4,2,health,services,...,2,2,1,1,5,2,15,14,15,30
4,GP,F,16,U,GT3,T,3,3,other,other,...,3,2,1,2,5,4,6,10,10,32


### 練習問題 6-8

In [542]:
# 分割の粒度
absences_bins = [0,1,5,100]
cut_absences = pd.cut(student_data_math.absences, absences_bins)

In [543]:
pd.value_counts(cut_absences)

/var/folders/5k/0bfc6pzn3z9_z6xn1rvv40sr0000gn/T/ipykernel_74642/1098367522.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(cut_absences)


absences
(5, 100]    146
(1, 5]      131
(0, 1]        3
Name: count, dtype: int64

### 練習問題 6-9

In [544]:
pd.value_counts(pd.qcut(student_data_math.absences, 3))

/var/folders/5k/0bfc6pzn3z9_z6xn1rvv40sr0000gn/T/ipykernel_74642/3802467107.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(pd.qcut(student_data_math.absences, 3))


absences
(-0.001, 2.0]    183
(6.0, 75.0]      115
(2.0, 6.0]        97
Name: count, dtype: int64

## 6-2-4 データの集約とグループ演算


In [545]:
# データを用意。regionつき
df1

,id,city,birth_year,name,region,up_two_num
0,100,Tokyo,1990,Hiroshi,Kanto,199
1,101,Osaka,1989,Akiko,NaN,198
2,102,Kyoto,1992,Yuki,Kansai,199
3,103,Hokkaido,1997,Satoru,Hokkaido,199
4,104,Tokyo,1982,Steeve,Kanto,198
5,106,Tokyo,1991,Mituru,Kanto,199
6,108,Osaka,1988,Aoi,NaN,198
7,110,Kyoto,1990,Tarou,Kansai,199
8,111,Hokkaido,1995,Suguru,Hokkaido,199
9,113,Tokyo,1981,Mitsuo,Kanto,198


In [546]:
# サイズ情報
df1.groupby('city').size()

city
Hokkaido    2
Kyoto       2
Osaka       2
Tokyo       4
dtype: int64

In [547]:
# cityを軸にbirth_yearの平均値を求める
df1.groupby('city')['birth_year'].mean()

city
Hokkaido    1996.0
Kyoto       1991.0
Osaka       1988.5
Tokyo       1986.0
Name: birth_year, dtype: float64

In [548]:
# 2軸での指定も可能
df1.groupby(['region', 'city'])['birth_year'].mean()

region    city    
Hokkaido  Hokkaido    1996.0
Kansai    Kyoto       1991.0
Kanto     Tokyo       1986.0
Name: birth_year, dtype: float64

In [549]:
# as_index=False パラメータの設定でインデックスが設定されなくなる
df1.groupby(['region', 'city'], as_index=False)['birth_year'].mean()

,region,city,birth_year
0,Hokkaido,Hokkaido,1996.0
1,Kansai,Kyoto,1991.0
2,Kanto,Tokyo,1986.0


In [550]:
# groupbyはイテレータで反復的に値を取り出すことも可能
for group, subdf in df1.groupby('region'):
	print('=======================================')
	print('Region name:{0}'.format(group))
	print(subdf)

Region name:Hokkaido
    id      city  birth_year    name    region up_two_num
3  103  Hokkaido        1997  Satoru  Hokkaido        199
8  111  Hokkaido        1995  Suguru  Hokkaido        199
Region name:Kansai
    id   city  birth_year   name  region up_two_num
2  102  Kyoto        1992   Yuki  Kansai        199
7  110  Kyoto        1990  Tarou  Kansai        199
Region name:Kanto
    id   city  birth_year     name region up_two_num
0  100  Tokyo        1990  Hiroshi  Kanto        199
4  104  Tokyo        1982   Steeve  Kanto        198
5  106  Tokyo        1991   Mituru  Kanto        199
9  113  Tokyo        1981   Mitsuo  Kanto        198


In [551]:
import pandas as pd

# 複数の計算をまとめて行うときはagg(aggregate:集約する)メソッドを使用
functions = ['count', 'mean', 'max', 'min']
grouped_student_math_data1 = student_data_math.groupby(['sex', 'address'])
result = grouped_student_math_data1.agg({'age': functions, 'G1': functions})

print(result)


              age                       G1                   
            count       mean max min count       mean max min
sex address                                                  
F   R          44  16.977273  19  15    44  10.295455  19   6
    U         164  16.664634  20  15   164  10.707317  18   4
M   R          44  17.113636  21  15    44  10.659091  18   3
    U         143  16.517483  22  15   143  11.405594  19   5


### 練習問題 6-10 

In [552]:
student_data_math.groupby('school')['G1'].mean()

school
GP    10.939828
MS    10.673913
Name: G1, dtype: float64

### 練習問題 6-11 

In [553]:
fs =['mean']
student_data_math.groupby(['school', 'sex']).agg({'G1':fs, 'G2':fs, 'G3':fs})

G1         G2         G3
                 mean       mean       mean
school sex                                 
GP     F    10.579235  10.398907   9.972678
       M    11.337349  11.204819  11.060241
MS     F    10.920000  10.320000   9.920000
       M    10.380952  10.047619   9.761905

### 練習問題 6-12 

In [554]:
fs = [max, min]
grouped_student = student_data_math.groupby(['school', 'sex'])
result = grouped_student.agg({'G1':fs, 'G2':fs, 'G3':fs})
result

/var/folders/5k/0bfc6pzn3z9_z6xn1rvv40sr0000gn/T/ipykernel_74642/550418783.py:3: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  result = grouped_student.agg({'G1':fs, 'G2':fs, 'G3':fs})
/var/folders/5k/0bfc6pzn3z9_z6xn1rvv40sr0000gn/T/ipykernel_74642/550418783.py:3: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  result = grouped_student.agg({'G1':fs, 'G2':fs, 'G3':fs})
/var/folders/5k/0bfc6pzn3z9_z6xn1rvv40sr0000gn/T/ipykernel_74642/550418783.py:3: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. 

G1      G2      G3    
           max min max min max min
school sex                        
GP     F    18   4  18   0  19   0
       M    19   3  19   0  20   0
MS     F    19   6  18   5  19   0
       M    15   6  16   5  16   0

# 6-3 欠損データと異常値の取り扱いの基礎
## 6-3-1 欠損データの扱い方

In [555]:
import numpy as np
from numpy import nan as NA
import pandas as pd
df = pd.DataFrame(np.random.rand(10,4))

# NAにする
# iloc (integer location)
df.iloc[1,0] = NA
df.iloc[2:3,2] = NA
df.iloc[5:,3] =NA
df

,0,1,2,3
0,0.626647,0.187629,0.469541,0.017204
1,NaN,0.274069,0.388218,0.224863
2,0.643475,0.285849,NaN,0.515964
3,0.459325,0.588574,0.741858,0.218504
4,0.428691,0.907517,0.624741,0.258119
5,0.036707,0.119595,0.422402,NaN
6,0.349591,0.027284,0.301087,NaN
7,0.015842,0.730633,0.728886,NaN
8,0.736277,0.380533,0.757876,NaN
9,0.415640,0.706828,0.609643,NaN


### 1-1 リストワイズ削除

In [556]:
# リストワイズ削除；NaNがある行を全て取り除く
df.dropna()

,0,1,2,3
0,0.626647,0.187629,0.469541,0.017204
3,0.459325,0.588574,0.741858,0.218504
4,0.428691,0.907517,0.624741,0.258119


### 1-2 ペアワイズ削除

In [557]:
# ペアワイズ削除：特定の列を取り出したのちに、そこにNaNがある行だけ削除
df[[0,1]].dropna()

,0,1
0,0.626647,0.187629
2,0.643475,0.285849
3,0.459325,0.588574
4,0.428691,0.907517
5,0.036707,0.119595
6,0.349591,0.027284
7,0.015842,0.730633
8,0.736277,0.380533
9,0.415640,0.706828


### 1-3 fillnaで埋める

In [558]:
# 特定の数字で置き換える
df.fillna(0)

,0,1,2,3
0,0.626647,0.187629,0.469541,0.017204
1,0.000000,0.274069,0.388218,0.224863
2,0.643475,0.285849,0.000000,0.515964
3,0.459325,0.588574,0.741858,0.218504
4,0.428691,0.907517,0.624741,0.258119
5,0.036707,0.119595,0.422402,0.000000
6,0.349591,0.027284,0.301087,0.000000
7,0.015842,0.730633,0.728886,0.000000
8,0.736277,0.380533,0.757876,0.000000
9,0.415640,0.706828,0.609643,0.000000


### 1-4 前の値で埋める

In [559]:
# forward fill methodで前の値で埋める。前に有効なデータがない場合は補完されない
df.fillna(method= 'ffill')

/var/folders/5k/0bfc6pzn3z9_z6xn1rvv40sr0000gn/T/ipykernel_74642/2912517256.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method= 'ffill')


,0,1,2,3
0,0.626647,0.187629,0.469541,0.017204
1,0.626647,0.274069,0.388218,0.224863
2,0.643475,0.285849,0.388218,0.515964
3,0.459325,0.588574,0.741858,0.218504
4,0.428691,0.907517,0.624741,0.258119
5,0.036707,0.119595,0.422402,0.258119
6,0.349591,0.027284,0.301087,0.258119
7,0.015842,0.730633,0.728886,0.258119
8,0.736277,0.380533,0.757876,0.258119
9,0.415640,0.706828,0.609643,0.258119


### 1-5 平均値で埋める

In [560]:
# 各カラムの平均値（確認よう）
df.mean()

0    0.412466
1    0.420851
2    0.560473
3    0.246931
dtype: float64

In [561]:
# 平均値で埋める
df.fillna(df.mean())

,0,1,2,3
0,0.626647,0.187629,0.469541,0.017204
1,0.412466,0.274069,0.388218,0.224863
2,0.643475,0.285849,0.560473,0.515964
3,0.459325,0.588574,0.741858,0.218504
4,0.428691,0.907517,0.624741,0.258119
5,0.036707,0.119595,0.422402,0.246931
6,0.349591,0.027284,0.301087,0.246931
7,0.015842,0.730633,0.728886,0.246931
8,0.736277,0.380533,0.757876,0.246931
9,0.415640,0.706828,0.609643,0.246931


### 練習問題 6-13

In [562]:
df2 = pd.DataFrame(np.random.rand(15,6))

# NAにする
# iloc (integer location)
df2.iloc[2,0] = NA
df2.iloc[5:8,2] = NA
df2.iloc[7:9,3] =NA
df2.iloc[10,5] =NA
df2

,0,1,2,3,4,5
0,0.821461,0.419525,0.623100,0.695708,0.517896,0.480954
1,0.618583,0.996891,0.563594,0.643411,0.287395,0.473040
2,NaN,0.700524,0.738991,0.742128,0.364481,0.889922
3,0.989483,0.414138,0.145643,0.358443,0.461639,0.666819
4,0.066264,0.524127,0.266972,0.193924,0.341245,0.209363
5,0.629540,0.884355,NaN,0.228916,0.475683,0.200923
6,0.564439,0.699731,NaN,0.654535,0.148517,0.559119
7,0.471161,0.414237,NaN,NaN,0.136249,0.670005
8,0.123717,0.324032,0.017199,NaN,0.715014,0.333697
9,0.642416,0.635406,0.773505,0.018473,0.183864,0.166809


In [563]:
df2.isnull()

,0,1,2,3,4,5
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,True,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False
5,False,False,True,False,False,False
6,False,False,True,False,False,False
7,False,False,True,True,False,False
8,False,False,False,True,False,False
9,False,False,False,False,False,False


In [564]:
df2.isnull().sum(axis=1)

0     0
1     0
2     1
3     0
4     0
5     1
6     1
7     2
8     1
9     0
10    1
11    0
12    0
13    0
14    0
dtype: int64

In [565]:
df2.isnull().sum(axis=0)

0    1
1    0
2    3
3    2
4    0
5    1
dtype: int64

In [566]:
df2.dropna()

,0,1,2,3,4,5
0,0.821461,0.419525,0.623100,0.695708,0.517896,0.480954
1,0.618583,0.996891,0.563594,0.643411,0.287395,0.473040
3,0.989483,0.414138,0.145643,0.358443,0.461639,0.666819
4,0.066264,0.524127,0.266972,0.193924,0.341245,0.209363
9,0.642416,0.635406,0.773505,0.018473,0.183864,0.166809
11,0.952457,0.956651,0.370837,0.724168,0.663823,0.409678
12,0.531181,0.207422,0.733282,0.317210,0.204895,0.067699
13,0.544232,0.608422,0.495505,0.944855,0.689277,0.359237
14,0.065894,0.235603,0.801161,0.919298,0.591031,0.461156


### 練習問題 6-14

In [567]:
df2.fillna(0)

,0,1,2,3,4,5
0,0.821461,0.419525,0.623100,0.695708,0.517896,0.480954
1,0.618583,0.996891,0.563594,0.643411,0.287395,0.473040
2,0.000000,0.700524,0.738991,0.742128,0.364481,0.889922
3,0.989483,0.414138,0.145643,0.358443,0.461639,0.666819
4,0.066264,0.524127,0.266972,0.193924,0.341245,0.209363
5,0.629540,0.884355,0.000000,0.228916,0.475683,0.200923
6,0.564439,0.699731,0.000000,0.654535,0.148517,0.559119
7,0.471161,0.414237,0.000000,0.000000,0.136249,0.670005
8,0.123717,0.324032,0.017199,0.000000,0.715014,0.333697
9,0.642416,0.635406,0.773505,0.018473,0.183864,0.166809


### 練習問題 6-15

In [568]:
df2.fillna(df2.mean())

,0,1,2,3,4,5
0,0.821461,0.419525,0.623100,0.695708,0.517896,0.480954
1,0.618583,0.996891,0.563594,0.643411,0.287395,0.473040
2,0.571052,0.700524,0.738991,0.742128,0.364481,0.889922
3,0.989483,0.414138,0.145643,0.358443,0.461639,0.666819
4,0.066264,0.524127,0.266972,0.193924,0.341245,0.209363
5,0.629540,0.884355,0.526340,0.228916,0.475683,0.200923
6,0.564439,0.699731,0.526340,0.654535,0.148517,0.559119
7,0.471161,0.414237,0.526340,0.556371,0.136249,0.670005
8,0.123717,0.324032,0.017199,0.556371,0.715014,0.333697
9,0.642416,0.635406,0.773505,0.018473,0.183864,0.166809


## 6-3-2 異常データの扱い方

# 6-4 時系列データの取り扱いの基礎

In [569]:
import pandas_datareader.data as pdr

## 6-4-1 時系列データの処理と変換

In [570]:
start_date = '2001/1/2'
end_date = '2016/12/30'

# FRED（Federal Reserve Economic Data）から取得できるドル円のレートを取得
fx_jpusdata = pdr.DataReader('DEXJPUS', 'fred', start_date, end_date)
fx_jpusdata.head()


,DEXJPUS
DATE,
2001-01-02,114.73
2001-01-03,114.26
2001-01-04,115.47
2001-01-05,116.19
2001-01-08,115.97


### 1-1 特定の年月のデータを参照する

In [571]:
fx_jpusdata.loc['2016-04']

,DEXJPUS
DATE,
2016-04-01,112.06
2016-04-04,111.18
2016-04-05,110.26
2016-04-06,109.63
2016-04-07,107.98
2016-04-08,108.36
2016-04-11,107.96
2016-04-12,108.54
2016-04-13,109.21


In [572]:
# resample()でデータを再集計、last()で末尾のデータを抽出
fx_jpusdata.resample('M').last().head()

/var/folders/5k/0bfc6pzn3z9_z6xn1rvv40sr0000gn/T/ipykernel_74642/3548363546.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  fx_jpusdata.resample('M').last().head()


,DEXJPUS
DATE,
2001-01-31,116.39
2001-02-28,117.28
2001-03-31,125.54
2001-04-30,123.57
2001-05-31,118.88


### 1-2 欠損がある場合の操作

In [573]:
fx_jpusdata.resample('D').last().head()

,DEXJPUS
DATE,
2001-01-02,114.73
2001-01-03,114.26
2001-01-04,115.47
2001-01-05,116.19
2001-01-06,NaN


In [574]:
# 前の日のデータで埋める処理を行う
fx_jpusdata.resample('D').ffill().head()

,DEXJPUS
DATE,
2001-01-02,114.73
2001-01-03,114.26
2001-01-04,115.47
2001-01-05,116.19
2001-01-06,116.19


### 1-3 データをずらして比率を計算する

In [575]:
# データを1日後ろにずらす
fx_jpusdata.shift(1).head()

,DEXJPUS
DATE,
2001-01-02,NaN
2001-01-03,114.73
2001-01-04,114.26
2001-01-05,115.47
2001-01-08,116.19


In [576]:
# 前日と当日のレートの比率を計算
fx_jpusdata_ratio = fx_jpusdata / fx_jpusdata.shift(1)
fx_jpusdata_ratio

,DEXJPUS
DATE,
2001-01-02,NaN
2001-01-03,0.995903
2001-01-04,1.010590
2001-01-05,1.006235
2001-01-08,0.998107
...,...
2016-12-26,NaN
2016-12-27,NaN
2016-12-28,1.001191


In [577]:
# diffメソッドで前の日のレートとの差分を計算
fx_jpusdata_diff = fx_jpusdata.diff()
fx_jpusdata_diff

,DEXJPUS
DATE,
2001-01-02,NaN
2001-01-03,-0.47
2001-01-04,1.21
2001-01-05,0.72
2001-01-08,-0.22
...,...
2016-12-26,NaN
2016-12-27,NaN
2016-12-28,0.14


In [578]:
# pct_changeメソッドで前日のレートとの比率をパーセント表示で求める
fx_jpusdata_pct = fx_jpusdata.pct_change()
fx_jpusdata_pct

/var/folders/5k/0bfc6pzn3z9_z6xn1rvv40sr0000gn/T/ipykernel_74642/1838222075.py:2: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  fx_jpusdata_pct = fx_jpusdata.pct_change()


,DEXJPUS
DATE,
2001-01-02,NaN
2001-01-03,-0.004097
2001-01-04,0.010590
2001-01-05,0.006235
2001-01-08,-0.001893
...,...
2016-12-26,0.000000
2016-12-27,0.002559
2016-12-28,0.001191


## 6-4-2 移動平均

In [579]:
fx_jpusdata.head()

,DEXJPUS
DATE,
2001-01-02,114.73
2001-01-03,114.26
2001-01-04,115.47
2001-01-05,116.19
2001-01-08,115.97


In [580]:
# 3日間の移動平均を計算
fx_jpusdata.rolling(3).mean().head()

,DEXJPUS
DATE,
2001-01-02,NaN
2001-01-03,NaN
2001-01-04,114.820000
2001-01-05,115.306667
2001-01-08,115.876667


In [581]:
# 3日間の標準偏差の推移を算出したいときはstdメソッド
fx_jpusdata.rolling(3).std().head()

,DEXJPUS
DATE,
2001-01-02,NaN
2001-01-03,NaN
2001-01-04,0.610000
2001-01-05,0.975312
2001-01-08,0.368963


### 練習問題 6-17

In [582]:
fx_jpusdata.rolling(20).mean().dropna()

,DEXJPUS
DATE,
2001-02-12,116.6910
2001-02-13,116.6920
2001-02-14,116.6070
2001-02-15,116.5015
2001-02-16,116.4130
...,...
2016-11-08,104.1600
2016-11-09,104.1780
2016-11-10,104.3250


## 6章　総合問題
### 総合問題 6-1 データ操作

In [583]:
student_data_math.pivot_table(values='G1', index='age', columns='sex', aggfunc='mean')

sex,F,M
age,,
15,10.052632,12.250000
16,10.203704,11.740000
17,11.103448,10.600000
18,10.883721,10.538462
19,10.642857,9.700000
20,15.000000,13.000000
21,NaN,10.000000
22,NaN,6.000000


In [584]:
student_data_math.pivot_table(values='G1', index='age', columns='sex', aggfunc='mean').dropna()

sex,F,M
age,,
15,10.052632,12.250000
16,10.203704,11.740000
17,11.103448,10.600000
18,10.883721,10.538462
19,10.642857,9.700000
20,15.000000,13.000000
